In [1]:
import findspark
findspark.init()
findspark.find()

import pyspark
findspark.find()

'C:\\BigDataLocalSetup\\spark'

In [2]:
import pyspark

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[2]") \
    .appName("Recommendation_System_GoodBooks") \
    .getOrCreate()

sc = spark.sparkContext

In [3]:
spark

In [4]:
# Load Dataset
data = spark.read.format('com.databricks.spark.csv').\
                       options(header='true', \
                       inferschema='true').\
            load("book_tags.csv",header=True);

# Data Preparation and Exploratory Data Analysis (EDA)

In [5]:
#import to_read data

file_location = "C:/Users/ITD-STU/PDB/Proyek/to_read.csv"
file_type = "csv"

# CSV optiAons
infer_schema = "True"
first_row_is_header = "True"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
to_read = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(to_read)

# Create Temporary Tables
to_read.createOrReplaceTempView("to_read")

DataFrame[user_id: int, book_id: int]

In [6]:
to_read.show(5)
to_read.printSchema()

+-------+-------+
|user_id|book_id|
+-------+-------+
|      1|    112|
|      1|    235|
|      1|    533|
|      1|   1198|
|      1|   1874|
+-------+-------+
only showing top 5 rows

root
 |-- user_id: integer (nullable = true)
 |-- book_id: integer (nullable = true)



In [7]:
#import ratings data
file_location = "C:/Users/ITD-STU/PDB/Proyek/ratings.csv"
file_type = "csv"

# CSV options
infer_schema = "True"
first_row_is_header = "True"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
ratings = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(ratings)

# Create Temporary Tables
ratings.createOrReplaceTempView("ratings")

DataFrame[book_id: int, user_id: int, rating: int]

In [8]:
ratings.show(5)
ratings.printSchema()

+-------+-------+------+
|book_id|user_id|rating|
+-------+-------+------+
|      1|    314|     5|
|      1|    439|     3|
|      1|    588|     5|
|      1|   1169|     4|
|      1|   1185|     4|
+-------+-------+------+
only showing top 5 rows

root
 |-- book_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- rating: integer (nullable = true)



In [9]:
#import books data
file_location = "C:/Users/ITD-STU/PDB/Proyek/books.csv"
file_type = "csv"

# CSV options
infer_schema = "True"
first_row_is_header = "True"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
books = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sto_readep", delimiter) \
  .load(file_location)

display(books)

# Create Temporary Tables
books.createOrReplaceTempView("books")

DataFrame[id: int, book_id: int, best_book_id: int, work_id: int, books_count: int, isbn: string, isbn13: double, authors: string, original_publication_year: double, original_title: string, title: string, language_code: string, average_rating: string, ratings_count: string, work_ratings_count: string, work_text_reviews_count: string, ratings_1: double, ratings_2: int, ratings_3: int, ratings_4: int, ratings_5: int, image_url: string, small_image_url: string]

In [10]:
books.show(5)
books.printSchema()

+---+-------+------------+-------+-----------+---------+----------------+--------------------+-------------------------+--------------------+--------------------+-------------+--------------+-------------+------------------+-----------------------+---------+---------+---------+---------+---------+--------------------+--------------------+
| id|book_id|best_book_id|work_id|books_count|     isbn|          isbn13|             authors|original_publication_year|      original_title|               title|language_code|average_rating|ratings_count|work_ratings_count|work_text_reviews_count|ratings_1|ratings_2|ratings_3|ratings_4|ratings_5|           image_url|     small_image_url|
+---+-------+------------+-------+-----------+---------+----------------+--------------------+-------------------------+--------------------+--------------------+-------------+--------------+-------------+------------------+-----------------------+---------+---------+---------+---------+---------+--------------------

In [11]:
#converting books data into pandas dataframe
books_df = books.toPandas()
books_df.head()

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715.0,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504.0,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191.0,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427.0,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236.0,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [12]:
import pandas as pd
import numpy as np

books_df = books_df[books_df.ratings_count.str.isdigit() == True]

In [13]:
books_df.ratings_count = books_df.ratings_count.astype('int')

In [14]:
#Top books with most number of ratings on goodbooks

books_df.sort_values(by = 'ratings_count', ascending = False)[['original_title','ratings_count', 'average_rating' ]][0:10]

,original_title,ratings_count,average_rating
0,The Hunger Games,4780653,4.34
1,Harry Potter and the Philosopher's Stone,4602479,4.44
2,Twilight,3866839,3.57
3,To Kill a Mockingbird,3198671,4.25
4,The Great Gatsby,2683664,3.89
5,The Fault in Our Stars,2346404,4.26
6,The Hobbit or There and Back Again,2071616,4.25
7,The Catcher in the Rye,2044241,3.79
9,Pride and Prejudice,2035490,4.24
8,Angels & Demons,2001311,3.85


In [15]:
most_ratings = books_df.sort_values(by = 'ratings_count', ascending = False)[['original_title','ratings_count', 'average_rating', 'image_url' ]][0:10]


#### Printing top books with most number of ratings

In [16]:
from IPython.display import Image, HTML

most_ratings['img_html'] = most_ratings['image_url']\
    .str.replace(
        '(.*)', 
        '<img src="\\1" style="max-height:124px;"></img>'
    )
with pd.option_context('display.max_colwidth', 10000):
  
  display(HTML(most_ratings[['original_title', 'img_html', 'ratings_count', 'average_rating' ]].to_html(escape=False)))

,original_title,img_html,ratings_count,average_rating
0,The Hunger Games,,4780653,4.34
1,Harry Potter and the Philosopher's Stone,,4602479,4.44
2,Twilight,,3866839,3.57
3,To Kill a Mockingbird,,3198671,4.25
4,The Great Gatsby,,2683664,3.89
5,The Fault in Our Stars,,2346404,4.26
6,The Hobbit or There and Back Again,,2071616,4.25
7,The Catcher in the Rye,,2044241,3.79
9,Pride and Prejudice,,2035490,4.24
8,Angels & Demons,,2001311,3.85


In [17]:
books_df.average_rating = books_df.average_rating.astype('float')

In [18]:
#Top books with top average ratings on goodbooks

books_df.sort_values(by = 'average_rating', ascending = False)[['original_title','ratings_count', 'average_rating' ]][0:10]

,original_title,ratings_count,average_rating
3627,The Complete Calvin and Hobbes,28900,4.82
861,Words of Radiance,73572,4.77
3274,None,33220,4.77
8853,Mark of the Lion Trilogy,9081,4.76
7946,None,8953,4.76
4482,It's a Magical World: A Calvin and Hobbes Coll...,22351,4.75
6360,There's Treasure Everywhere: A Calvin and Hobb...,16766,4.74
421,Complete Harry Potter Boxed Set,190050,4.74
6589,The Authoritative Calvin and Hobbes,16087,4.73
6919,The Indispensable Calvin and Hobbes: A Calvin ...,14597,4.73


In [19]:
high_rating_books = books_df.sort_values(by = 'average_rating', ascending = False)[['original_title','ratings_count','image_url', 'average_rating' ]][0:10]

#### Printing top books with highest average ratings

In [20]:
high_rating_books['img_html'] = high_rating_books['image_url']\
    .str.replace(
        '(.*)', 
        '<img src="\\1" style="max-height:124px;"></img>'
    )
with pd.option_context('display.max_colwidth', 10000):
  
  display(HTML(high_rating_books[['original_title', 'img_html','ratings_count', 'average_rating' ]].to_html(escape=False)))

,original_title,img_html,ratings_count,average_rating
3627,The Complete Calvin and Hobbes,,28900,4.82
861,Words of Radiance,,73572,4.77
3274,None,,33220,4.77
8853,Mark of the Lion Trilogy,,9081,4.76
7946,None,,8953,4.76
4482,It's a Magical World: A Calvin and Hobbes Collection,,22351,4.75
6360,There's Treasure Everywhere: A Calvin and Hobbes Collection,,16766,4.74
421,Complete Harry Potter Boxed Set,,190050,4.74
6589,The Authoritative Calvin and Hobbes,,16087,4.73
6919,The Indispensable Calvin and Hobbes: A Calvin and Hobbes Treasury,,14597,4.73


In [21]:
authors_with_most_books = pd.DataFrame(books_df.authors.value_counts()[0:10]).reset_index()
authors_with_most_books.columns = ['author', 'number_of_books']

In [22]:
authors_with_most_books

,author,number_of_books
0,Stephen King,60
1,Nora Roberts,59
2,Dean Koontz,47
3,Terry Pratchett,42
4,Agatha Christie,39
5,Meg Cabot,37
6,James Patterson,36
7,David Baldacci,34
8,John Grisham,33
9,J.D. Robb,33


In [23]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,6))
plt.title('Distribution of Average Ratings')
books_df['average_rating'].hist()
display()

In [24]:
ratings.show(5)

+-------+-------+------+
|book_id|user_id|rating|
+-------+-------+------+
|      1|    314|     5|
|      1|    439|     3|
|      1|    588|     5|
|      1|   1169|     4|
|      1|   1185|     4|
+-------+-------+------+
only showing top 5 rows



In [25]:
# Use .printSchema() to see the datatypes of the ratings dataset
ratings.printSchema()

root
 |-- book_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- rating: integer (nullable = true)



In [26]:
# Convert the columns to the proper data types
ratings = ratings.select(ratings.user_id,
                         ratings.book_id,
                         ratings.rating.cast("double"))

In [27]:
# Call .printSchema() again to confirm the columns are now in the correct format
ratings.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- book_id: integer (nullable = true)
 |-- rating: double (nullable = true)



In [28]:
ratings.show(5)

+-------+-------+------+
|user_id|book_id|rating|
+-------+-------+------+
|    314|      1|   5.0|
|    439|      1|   3.0|
|    588|      1|   5.0|
|   1169|      1|   4.0|
|   1185|      1|   4.0|
+-------+-------+------+
only showing top 5 rows



In [29]:
# Menghitung total number of ratings pada dataset
numerator = ratings.select("rating").count()

# Menghitung number of distinct Id's
num_users = ratings.select("user_id").distinct().count()
num_items = ratings.select("book_id").distinct().count()

# Denominator sama dengan number of users multiplied berdasarkan number of items
denominator = num_users * num_items

sparsity = (1.0 - (numerator * 1.0)/ denominator) * 100
print("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

The ratings dataframe is  99.82% empty.


In [30]:
# The GroupBy and Filter Methods
# Exploratory Data Analysis (EDA)
from pyspark.sql.functions import col, min, max, avg, lit

# Group data by user_id, count ratings
(ratings.groupBy("user_id")
    .count()
    .filter("`count` > 1")
    .sort(col("count").desc())
    .show(n = 10))

+-------+-----+
|user_id|count|
+-------+-----+
|  12874|  200|
|  30944|  200|
|  28158|  199|
|  12381|  199|
|  52036|  199|
|   6630|  197|
|  37834|  197|
|  45554|  197|
|   9806|  196|
|  19729|  196|
+-------+-----+
only showing top 10 rows



In [31]:
# Group data by book_id, count ratings
(ratings.groupBy("book_id")
    .count()
    .filter("`count` > 1")
    .sort(col("count").desc())
    .show(n = 10))

+-------+-----+
|book_id|count|
+-------+-----+
|    148|  100|
|    463|  100|
|    471|  100|
|    496|  100|
|    833|  100|
|   1088|  100|
|   1238|  100|
|   1342|  100|
|   1580|  100|
|   1591|  100|
+-------+-----+
only showing top 10 rows



In [32]:
# Group data berdasarkan book_id dan count ratings
(ratings.groupBy("book_id")
    .count()
    .filter("`count` > 1")
    .sort(col("count").desc())
    .show(n = 10))

+-------+-----+
|book_id|count|
+-------+-----+
|    243|  100|
|    392|  100|
|    540|  100|
|    623|  100|
|    737|  100|
|    858|  100|
|    897|  100|
|   1025|  100|
|   1084|  100|
|   1127|  100|
+-------+-----+
only showing top 10 rows



#### Summary Statistics

In [33]:
# Min num ratings
print("Item dengan peringkat (rating) paling sedikit: ")
ratings.groupBy("book_id").count().select(min("count")).show()

Item dengan peringkat (rating) paling sedikit: 
+----------+
|min(count)|
+----------+
|         8|
+----------+



In [34]:
# Avg num ratings 
print("Nilai rata-rata rating per item: ")
ratings.groupBy("book_id").count().select(avg("count")).show()

Nilai rata-rata rating per item: 
+----------+
|avg(count)|
+----------+
|   98.1756|
+----------+



In [35]:
# Min num ratings untuk pengguna
print("Pengguna dengan peringkat (rating) paling sedikit: ")
ratings.groupBy("user_id").count().select(min("count")).show()

Pengguna dengan peringkat (rating) paling sedikit: 
+----------+
|min(count)|
+----------+
|         2|
+----------+



In [36]:
# Min num ratings 
print("Item dengan peringkat (rating) paling sedikit: ")
ratings.groupBy("book_id").count().sort('count').show(10)

Item dengan peringkat (rating) paling sedikit: 
+-------+-----+
|book_id|count|
+-------+-----+
|   7803|    8|
|   9345|   11|
|   9486|   24|
|   1935|   34|
|   9315|   36|
|   9553|   41|
|   8181|   44|
|   9260|   46|
|   9479|   47|
|   9705|   48|
+-------+-----+
only showing top 10 rows



In [37]:
# Avg num ratings per users
print("Nilai rata-rata rating per pengguna: ")
ratings.groupBy("user_id").count().select(avg("count")).show()

Nilai rata-rata rating per pengguna: 
+------------------+
|        avg(count)|
+------------------+
|18.376684636118597|
+------------------+



In [38]:
#1 million ratings
ratings.describe().show()

+-------+------------------+-----------------+------------------+
|summary|           user_id|          book_id|            rating|
+-------+------------------+-----------------+------------------+
|  count|            981756|           981756|            981756|
|   mean|25616.759933221696|4943.275635697668|3.8565335989797873|
| stddev|15228.338825882167|2873.207414896114|0.9839408559620033|
|    min|                 1|                1|               1.0|
|    max|             53424|            10000|               5.0|
+-------+------------------+-----------------+------------------+



In [39]:
# Buku yang memeilikki 100 rating

ratings.groupby('book_id').count().show()

+-------+-----+
|book_id|count|
+-------+-----+
|    148|  100|
|    463|  100|
|    471|  100|
|    496|  100|
|    833|  100|
|   1088|  100|
|   1238|  100|
|   1342|  100|
|   1580|  100|
|   1591|  100|
|   1645|  100|
|   1829|  100|
|   1959|  100|
|   2122|  100|
|   2142|  100|
|   2366|  100|
|   2659|  100|
|   2866|  100|
|   3175|  100|
|   3749|  100|
+-------+-----+
only showing top 20 rows



# MACHINE LEARNING (ML)

Proyek ini akan menggunakan model ALS untuk memfaktorkan matriks user-book menjadi dua matriks terpisah. Ketika balik ke proses ini dengan mengalikan dua faktor matriks, maka akan mendapatkan perkiraan untuk setiap sel di matriks awal juga sel-sel yang kosong. Dimana setiap sel akan diisi sengan angka. Nilai yang ada di matriks awalakan bertindak sebagai label untuk training proses. Model tersebut akan meminimalkan perbedaan antara matriks faktor yang artinya akan memprediksi nilai-nilai yang kosong.

Algoritma faktorisasi akan menjadi non-negative matrix factorization (NMF atau NNMF), karena yang kita inginkan hanyalah nilai yang positif. Pada goodreads book rating sistem akan merating nilai postif dengan rentang 0-5.

In [40]:
(training, test) = ratings.randomSplit([0.8, 0.2])

In [41]:
test.show(5)

+-------+-------+------+
|user_id|book_id|rating|
+-------+-------+------+
|      1|   1180|   4.0|
|      1|   4893|   3.0|
|      1|   6285|   4.0|
|      7|   1199|   4.0|
|      7|   1464|   5.0|
+-------+-------+------+
only showing top 5 rows



In [42]:
# Machine Learning (ML)
from pyspark.ml.recommendation import ALS # Alternating Least Squares model
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator # Cross-Validation
from pyspark.ml.evaluation import RegressionEvaluator # Performance metric

#### Cross Validation Pipeline

In [43]:
# Membuat Generic ALS model - tanpa hyperparameters
als = ALS(userCol="user_id", itemCol="book_id", ratingCol="rating", 
          nonnegative = True, # Non negative matrix factorization
          coldStartStrategy = "drop", # What to do if user do not appear in train and test set
          implicitPrefs = False) # Explicit preference

In [44]:
# Confirm that a model called "als" was created
type(als)

pyspark.ml.recommendation.ALS

##### Train test split

In [45]:
# Membuat test and train set
(train, test) = ratings.randomSplit([0.8, 0.2], 
                                    seed = 1234)
print(type(train))

<class 'pyspark.sql.dataframe.DataFrame'>


#### Hyperparameter Grid

In [46]:
# Menambah hyperparameters dan respective values menjadi param_grid
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50, 100]) \
            .addGrid(als.maxIter, [5, 50, 100]) \
            .addGrid(als.regParam, [.01, .05, .1]) \
            .build()

#### Performance metric

In [47]:
# Define evaluator as RMSE
evaluator = RegressionEvaluator(metricName = "rmse", 
                                labelCol = "rating", 
                                predictionCol = "prediction")
# Print length of evaluator
print ("Num models to be tested: ", len(param_grid))

Num models to be tested:  27


#### Cross validation 

In [48]:
# Build cross validation using CrossValidator
CV = CrossValidator(estimator = als, 
                    estimatorParamMaps = param_grid, 
                    evaluator = evaluator, 
                    numFolds = 5)
# Confirm cv was built
print(CV)

CrossValidator_4c9cc702d762


### Alternating Least Squares Model (ALS)

#### Generic model

In [49]:
# Fit generic model to the 'train' dataset
als_mod = als.fit(train)
test_pred = als_mod.transform(test)

# View the predictions 
test_pred.show(n = 10)

+-------+-------+------+----------+
|user_id|book_id|rating|prediction|
+-------+-------+------+----------+
|  20467|    148|   3.0|  2.988968|
|   3087|    148|   3.0|  3.490897|
|  10727|    148|   4.0|  3.331768|
|  41282|    148|   3.0| 4.0919285|
|  24326|    148|   5.0| 3.3628643|
|  32055|    148|   3.0| 3.1412587|
|  27834|    148|   3.0|  3.919591|
|  13991|    148|   4.0| 3.7761638|
|  15547|    148|   4.0|  2.939672|
|  14248|    148|   4.0| 3.5569577|
+-------+-------+------+----------+
only showing top 10 rows



In [50]:
# Calculate and print the RMSE of test_predictions
print(evaluator.evaluate(test_pred))

0.9189663667500532


#### Predictions and Performance Evaluation

In [51]:
best_model = als_mod

In [52]:
test_predictions = best_model.transform(test)

# View the predictions 
test_predictions.show(n = 10)

# Calculate and print the RMSE of test_predictions
print ("RMSE: ", evaluator.evaluate(test_predictions))

+-------+-------+------+----------+
|user_id|book_id|rating|prediction|
+-------+-------+------+----------+
|  20467|    148|   3.0|  2.988968|
|   3087|    148|   3.0|  3.490897|
|  10727|    148|   4.0|  3.331768|
|  41282|    148|   3.0| 4.0919285|
|  24326|    148|   5.0| 3.3628643|
|  32055|    148|   3.0| 3.1412587|
|  27834|    148|   3.0|  3.919591|
|  13991|    148|   4.0| 3.7761638|
|  15547|    148|   4.0|  2.939672|
|  14248|    148|   4.0| 3.5569577|
+-------+-------+------+----------+
only showing top 10 rows

RMSE:  0.9189663667500532


#### Model Performance Evaluation and Output Cleanup
 

In [53]:
# Generate n recommendations for all users
ALS_recommendations = best_model.recommendForAllUsers(numItems = 10) # n - 10

Berikut hasil 10 rekomendasi untuk semua pengguna. Ini juga dapat menghasilkan rekomendasi untuk buku yang telah dibaca pengguna.

In [54]:
ALS_recommendations.show(n = 10)

+-------+--------------------+
|user_id|     recommendations|
+-------+--------------------+
|    148|[[8946, 4.7048097...|
|    463|[[9550, 4.977393]...|
|    471|[[8616, 2.993556]...|
|    496|[[3358, 5.465655]...|
|    833|[[9842, 4.7741456...|
|   1088|[[8946, 4.917199]...|
|   1238|[[8946, 5.49808],...|
|   1342|[[2064, 5.073918]...|
|   1580|[[8946, 5.054802]...|
|   1591|[[3498, 3.1270342...|
+-------+--------------------+
only showing top 10 rows



In [55]:
# Temporary table
ALS_recommendations.registerTempTable("ALS_recs_temp")